In [47]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import date
import re
import numpy as np
import warnings
from icecream import ic

ic.configureOutput(includeContext=True)
warnings.filterwarnings("ignore")

In [48]:
# get the Data
def get_data(path, displ=False):
    ## import data
    df = pd.read_csv(path, parse_dates=['date'], infer_datetime_format=True)

    # sort data
    df = df.sort_values(by=["number_sta", "date"])

    # set number_sta as category
    df["number_sta"] = df["number_sta"].astype("category")
    if displ:
        display("df :", df.head())

    return df

In [49]:
def get_observations(x, displ=False):
    ## shift X
    #get the observation baseline
    obs = x[{"number_sta", "date", "precip"}]
    # obs.set_index('date', inplace=True)

    #if any NaN on the day, then the value is NaN (24 values per day)
    # obs = obs.groupby('number_sta').resample('D')#.agg(pd.Series.sum, min_count = 24)
    obs['date'] = obs['date'].astype('category')
    obs['number_sta'] = obs['number_sta'].astype('category')
    obs['baseline_obs'] = obs.groupby(['number_sta'])['precip'].shift(1)

    obs = obs.sort_values(by=["number_sta", "date"])
    del obs['precip']
    obs = obs.rename(columns={'baseline_obs': 'precip'})
    # obs_new = obs.reset_index()

    if displ:
        display(obs)

    return obs

In [54]:
x_df = get_data(path='../Train/Train/X_station_train.csv')
y_df = get_data(path='../Train/Train/Y_train.csv')

In [55]:
print(ic(x_df))
print(ic(y_df))

ic| 987194678.py:1 in <module>
    x_df:         number_sta                date    ff       t      td    hu     dd  \
          0         14066001 2016-01-01 00:00:00  3.05  279.28  277.97  91.4  200.0   
          1         14066001 2016-01-01 01:00:00  2.57  278.76  277.45  91.4  190.0   
          2         14066001 2016-01-01 02:00:00  2.26  278.27  277.02  91.7  181.0   
          3         14066001 2016-01-01 03:00:00  2.62  277.98  276.95  93.0  159.0   
          4         14066001 2016-01-01 04:00:00  2.99  277.32  276.72  95.9  171.0   
          ...            ...                 ...   ...     ...     ...   ...    ...   
          4409469   95690001 2017-12-30 19:00:00  9.10  286.68  283.44  80.8  239.0   
          4409470   95690001 2017-12-30 20:00:00  8.58  286.39  283.21  81.1  231.0   
          4409471   95690001 2017-12-30 21:00:00  8.74  286.28  283.40  82.6  226.0   
          4409472   95690001 2017-12-30 22:00:00  9.04  286.21  283.29  82.4  224.0   
          44

        number_sta                date    ff       t      td    hu     dd  \
0         14066001 2016-01-01 00:00:00  3.05  279.28  277.97  91.4  200.0   
1         14066001 2016-01-01 01:00:00  2.57  278.76  277.45  91.4  190.0   
2         14066001 2016-01-01 02:00:00  2.26  278.27  277.02  91.7  181.0   
3         14066001 2016-01-01 03:00:00  2.62  277.98  276.95  93.0  159.0   
4         14066001 2016-01-01 04:00:00  2.99  277.32  276.72  95.9  171.0   
...            ...                 ...   ...     ...     ...   ...    ...   
4409469   95690001 2017-12-30 19:00:00  9.10  286.68  283.44  80.8  239.0   
4409470   95690001 2017-12-30 20:00:00  8.58  286.39  283.21  81.1  231.0   
4409471   95690001 2017-12-30 21:00:00  8.74  286.28  283.40  82.6  226.0   
4409472   95690001 2017-12-30 22:00:00  9.04  286.21  283.29  82.4  224.0   
4409473   95690001 2017-12-30 23:00:00  9.11  285.92  282.42  79.4  221.0   

         precip               Id  
0           0.0     14066001_0_0  
1    

In [27]:
print("x_df :")
for col in x_df.columns :
    print(col,"\t",x_df[col].isna().sum())
    if x_df[col].isna().sum() :
        x_df[col].fillna(value=x_df[col].mean(), inplace=True)
print("\nAfter filling nans :","\n",x_df.isna().sum())

print("\n\ny_df :")
for col in y_df.columns :
    print(col,"\t",y_df[col].isna().sum())
    if y_df[col].isna().sum() :
        y_df[col].fillna(value=y_df[col].mean(), inplace=True)
print("\nAfter filling nans :","\n",y_df.isna().sum())

x_df :
number_sta 	 0
date 	 0
ff 	 1750817
t 	 231013
td 	 1428352
hu 	 1425877
dd 	 1752650
precip 	 310298
Id 	 0

After filling nans : 
 number_sta    0
date          0
ff            0
t             0
td            0
hu            0
dd            0
precip        0
Id            0
dtype: int64


y_df :
date 	 0
number_sta 	 0
Ground_truth 	 21640
Id 	 0

After filling nans : 
 date            0
number_sta      0
Ground_truth    0
Id              0
dtype: int64
x_df :
number_sta 	 0
date 	 0
ff 	 1750817
t 	 231013
td 	 1428352
hu 	 1425877
dd 	 1752650
precip 	 310298
Id 	 0

After filling nans : 
 number_sta    0
date          0
ff            0
t             0
td            0
hu            0
dd            0
precip        0
Id            0
dtype: int64


y_df :
date 	 0
number_sta 	 0
Ground_truth 	 21640
Id 	 0

After filling nans : 
 date            0
number_sta      0
Ground_truth    0
Id              0
dtype: int64


In [28]:
obs = get_observations(x_df)
for col in obs.columns :
    print(col,"\t",obs[col].isna().sum())
    if obs[col].isna().sum() :
        obs[col].fillna(value=obs[col].mean(), inplace=True)

print("\nAfter filling nans :","\n",obs.isna().sum())

number_sta 	 0
date 	 0
precip 	 267

After filling nans : 
 number_sta    0
date          0
precip        0
dtype: int64
number_sta 	 0
date 	 0
precip 	 267

After filling nans : 
 number_sta    0
date          0
precip        0
dtype: int64


In [29]:
x_clean = clean_data(x_df, ["date"])
x_df = clean_data(x_df)
obs_clean = clean_data(obs, ["date"])
display(x_df)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


In [30]:
obs_clean.reset_index(drop=True, inplace=True)
x_clean.reset_index(drop=True, inplace=True)
x_df.reset_index(drop=True, inplace=True)
na_ids = np.where(pd.isnull(obs_clean).any(1))[0]
print(na_ids)

[]
[]


In [31]:
obs_clean.dropna(inplace=True)
x_clean.drop(index=na_ids, inplace=True)
x_df.drop(index=na_ids, inplace=True)

obs_clean.reset_index(drop=True, inplace=True)
x_clean.reset_index(drop=True, inplace=True)
x_df.reset_index(drop=True, inplace=True)

display(obs_clean)
display(x_clean)
display(x_df)

,number_sta,precip
0,14066001,0.080944
1,14066001,0.000000
2,14066001,0.000000
3,14066001,0.000000
4,14066001,0.000000
...,...,...
4409469,95690001,0.000000
4409470,95690001,0.000000
4409471,95690001,0.000000
4409472,95690001,0.000000


,number_sta,ff,t,td,hu,dd,precip,Id
0,14066001,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...
4409469,95690001,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


,number_sta,precip
0,14066001,0.080944
1,14066001,0.000000
2,14066001,0.000000
3,14066001,0.000000
4,14066001,0.000000
...,...,...
4409469,95690001,0.000000
4409470,95690001,0.000000
4409471,95690001,0.000000
4409472,95690001,0.000000


,number_sta,ff,t,td,hu,dd,precip,Id
0,14066001,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...
4409469,95690001,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


In [32]:
# obs_clean.set_index("number_sta", inplace=True)
# obs_clean = obs_clean["precip"]
# x_clean.reset_index(drop=True, inplace=True)
# x_clean.set_index("number_sta", inplace=True)

In [33]:
# linear regression
lin_reg = LinearRegression()
lin_reg.fit(x_clean, obs_clean)
prediction = lin_reg.predict(x_clean)

In [34]:
prediction = pd.DataFrame(prediction, columns=["number_sta","precip"])
prediction.iloc[:,0] = np.array(prediction.iloc[:,0],dtype=int)
print(prediction)
# prediction

         number_sta    precip
0          14066001  0.056030
1          14066001  0.049153
2          14066001  0.045084
3          14066001  0.050650
4          14066001  0.061778
...             ...       ...
4409469    95690000  0.116877
4409470    95690000  0.111159
4409471    95690000  0.116764
4409472    95690000  0.119022
4409473    95690000  0.109963

[4409474 rows x 2 columns]
         number_sta    precip
0          14066001  0.056030
1          14066001  0.049153
2          14066001  0.045084
3          14066001  0.050650
4          14066001  0.061778
...             ...       ...
4409469    95690000  0.116877
4409470    95690000  0.111159
4409471    95690000  0.116764
4409472    95690000  0.119022
4409473    95690000  0.109963

[4409474 rows x 2 columns]


In [35]:
prediction["date"] = x_df["date"]
# prediction["Id"] = x_df["Id"]
print(prediction)

         number_sta    precip                date
0          14066001  0.056030 2016-01-01 00:00:00
1          14066001  0.049153 2016-01-01 01:00:00
2          14066001  0.045084 2016-01-01 02:00:00
3          14066001  0.050650 2016-01-01 03:00:00
4          14066001  0.061778 2016-01-01 04:00:00
...             ...       ...                 ...
4409469    95690000  0.116877 2017-12-30 19:00:00
4409470    95690000  0.111159 2017-12-30 20:00:00
4409471    95690000  0.116764 2017-12-30 21:00:00
4409472    95690000  0.119022 2017-12-30 22:00:00
4409473    95690000  0.109963 2017-12-30 23:00:00

[4409474 rows x 3 columns]
         number_sta    precip                date
0          14066001  0.056030 2016-01-01 00:00:00
1          14066001  0.049153 2016-01-01 01:00:00
2          14066001  0.045084 2016-01-01 02:00:00
3          14066001  0.050650 2016-01-01 03:00:00
4          14066001  0.061778 2016-01-01 04:00:00
...             ...       ...                 ...
4409469    95690000  0

In [44]:
prediction.set_index('date',inplace = True)
ic(prediction)

KeyError: "None of ['date'] are in the columns"

In [37]:
prediction_summed = prediction.groupby('number_sta').resample('D').agg(pd.Series.sum, min_count = 24)

In [38]:
display(prediction_summed)

number_sta    precip
number_sta date                              
14066001   2016-01-01  3.375840e+08  1.436778
           2016-01-02  3.375840e+08  3.595443
           2016-01-03  3.375840e+08  6.567125
           2016-01-04  3.375840e+08  1.888832
           2016-01-05  3.375840e+08  1.764836
...                             ...       ...
95690000   2017-12-26  2.296560e+09  3.217714
           2017-12-27  2.296560e+09  3.670035
           2017-12-28  2.296560e+09  1.003673
           2017-12-29  2.296560e+09  3.694025
           2017-12-30  2.296560e+09  5.251946

[191134 rows x 2 columns]

number_sta    precip
number_sta date                              
14066001   2016-01-01  3.375840e+08  1.436778
           2016-01-02  3.375840e+08  3.595443
           2016-01-03  3.375840e+08  6.567125
           2016-01-04  3.375840e+08  1.888832
           2016-01-05  3.375840e+08  1.764836
...                             ...       ...
95690000   2017-12-26  2.296560e+09  3.217714
           2017-12-27  2.296560e+09  3.670035
           2017-12-28  2.296560e+09  1.003673
           2017-12-29  2.296560e+09  3.694025
           2017-12-30  2.296560e+09  5.251946

[191134 rows x 2 columns]

In [39]:
p = pd.DataFrame(prediction_summed)
del p["number_sta"]
p = p.reset_index()

p["date"] = pd.to_datetime(p["date"])

# p.columns = ['number_sta', 'date', 'precip']

p['number_sta'] = p['number_sta'].astype('category')

display(p)

,number_sta,date,precip
0,14066001,2016-01-01,1.436778
1,14066001,2016-01-02,3.595443
2,14066001,2016-01-03,6.567125
3,14066001,2016-01-04,1.888832
4,14066001,2016-01-05,1.764836
...,...,...,...
191129,95690000,2017-12-26,3.217714
191130,95690000,2017-12-27,3.670035
191131,95690000,2017-12-28,1.003673
191132,95690000,2017-12-29,3.694025


,number_sta,date,precip
0,14066001,2016-01-01,1.436778
1,14066001,2016-01-02,3.595443
2,14066001,2016-01-03,6.567125
3,14066001,2016-01-04,1.888832
4,14066001,2016-01-05,1.764836
...,...,...,...
191129,95690000,2017-12-26,3.217714
191130,95690000,2017-12-27,3.670035
191131,95690000,2017-12-28,1.003673
191132,95690000,2017-12-29,3.694025


In [40]:
print(y_df)

             date number_sta  Ground_truth            Id
0      2016-01-02   14066001           3.4    14066001_0
249    2016-01-03   14066001          11.7    14066001_1
499    2016-01-04   14066001           0.6    14066001_2
749    2016-01-05   14066001           0.4    14066001_3
997    2016-01-06   14066001           3.0    14066001_4
...           ...        ...           ...           ...
182747 2017-12-27   95690001           3.2  95690001_725
182997 2017-12-28   95690001           0.0  95690001_726
183247 2017-12-29   95690001           4.4  95690001_727
183497 2017-12-30   95690001           5.4  95690001_728
183746 2017-12-31   95690001           1.2  95690001_729

[183747 rows x 4 columns]
             date number_sta  Ground_truth            Id
0      2016-01-02   14066001           3.4    14066001_0
249    2016-01-03   14066001          11.7    14066001_1
499    2016-01-04   14066001           0.6    14066001_2
749    2016-01-05   14066001           0.4    14066001_3
997 

In [41]:
y_df.isna().sum(axis=0)

date            0
number_sta      0
Ground_truth    0
Id              0
dtype: int64

date            0
number_sta      0
Ground_truth    0
Id              0
dtype: int64

In [42]:
p.to_csv("prediction-ish.csv")

In [42]:
"""

merge le X_test avec la baseline en utilisant l'id. Supprimer l'heure, puis sommer sur le jours. pd.merge() ; pd.DataFrame.add()

faire un merge how="inner" entre baseline et prédiction, sur l'Id

"""